In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile1 = pd.read_pickle("BBB_Train_Decile1.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile1.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile1, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 1')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 3s 907us/step - loss: 0.5573 - acc: 0.7347 - val_loss: 0.5287 - val_acc: 0.7500
Epoch 38/100
3087/3087 [==============================] - 3s 917us/step - loss: 0.5556 - acc: 0.7305 - val_loss: 0.5282 - val_acc: 0.7526
Epoch 39/100
3087/3087 [==============================] - 3s 892us/step - loss: 0.5502 - acc: 0.7373 - val_loss: 0.5269 - val_acc: 0.7500
Epoch 40/100
3087/3087 [==============================] - 3s 929us/step - loss: 0.5554 - acc: 0.7340 - val_loss: 0.5265 - val_acc: 0.7526
Epoch 41/100
3087/3087 [==============================] - 3s 857us/step - loss: 0.5561 - acc: 0.7347 - val_loss: 0.5263 - val_acc: 0.7500
Epoch 42/100
3087/3087 [==============================] - 2s 770us/step - loss: 0.5529 - acc: 0.7386 - val_loss: 0.5255 - val_acc: 0.7500
Epoch 43/100
3087/3087 [==============================] - 2s 504us/step - loss: 0.5538 - acc: 0.7373 - val_loss: 0.5247 - val_acc: 0.7500
Epoch 44/100
3087/3087 [=======================

3087/3087 [==============================] - 2s 752us/step - loss: 0.5466 - acc: 0.7392 - val_loss: 0.5208 - val_acc: 0.7487
Epoch 97/100
3087/3087 [==============================] - 2s 755us/step - loss: 0.5437 - acc: 0.7402 - val_loss: 0.5208 - val_acc: 0.7487
Epoch 98/100
3087/3087 [==============================] - 2s 738us/step - loss: 0.5456 - acc: 0.7373 - val_loss: 0.5207 - val_acc: 0.7487
Epoch 99/100
3087/3087 [==============================] - 2s 756us/step - loss: 0.5474 - acc: 0.7405 - val_loss: 0.5208 - val_acc: 0.7474
Epoch 100/100
3087/3087 [==============================] - 2s 767us/step - loss: 0.5435 - acc: 0.7383 - val_loss: 0.5208 - val_acc: 0.7474
Test accuracy: 0.7474093264248705
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.6371 - acc: 0.6284 - val_loss: 0.5845 - val_acc: 0.6878
Epoch 2/150
3087/3087 [==============================] - 2s 800us/step - loss: 0.5932 - acc: 0.7000 - val_

3087/3087 [==============================] - 3s 955us/step - loss: 0.5496 - acc: 0.7434 - val_loss: 0.5197 - val_acc: 0.7565
Epoch 54/150
3087/3087 [==============================] - 3s 898us/step - loss: 0.5456 - acc: 0.7438 - val_loss: 0.5197 - val_acc: 0.7552
Epoch 55/150
3087/3087 [==============================] - 3s 854us/step - loss: 0.5460 - acc: 0.7428 - val_loss: 0.5197 - val_acc: 0.7552
Epoch 56/150
3087/3087 [==============================] - 3s 875us/step - loss: 0.5452 - acc: 0.7486 - val_loss: 0.5196 - val_acc: 0.7578
Epoch 57/150
3087/3087 [==============================] - 3s 851us/step - loss: 0.5426 - acc: 0.7402 - val_loss: 0.5196 - val_acc: 0.7578
Epoch 58/150
3087/3087 [==============================] - 3s 842us/step - loss: 0.5493 - acc: 0.7405 - val_loss: 0.5198 - val_acc: 0.7539
Epoch 59/150
3087/3087 [==============================] - 3s 876us/step - loss: 0.5468 - acc: 0.7421 - val_loss: 0.5197 - val_acc: 0.7552
Epoch 60/150
3087/3087 [=======================

3087/3087 [==============================] - 3s 909us/step - loss: 0.5421 - acc: 0.7431 - val_loss: 0.5193 - val_acc: 0.7565
Epoch 113/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5405 - acc: 0.7415 - val_loss: 0.5192 - val_acc: 0.7578
Epoch 114/150
3087/3087 [==============================] - 3s 954us/step - loss: 0.5435 - acc: 0.7457 - val_loss: 0.5193 - val_acc: 0.7565
Epoch 115/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5433 - acc: 0.7447 - val_loss: 0.5193 - val_acc: 0.7565
Epoch 116/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5433 - acc: 0.7425 - val_loss: 0.5194 - val_acc: 0.7539
Epoch 117/150
3087/3087 [==============================] - 3s 960us/step - loss: 0.5427 - acc: 0.7434 - val_loss: 0.5193 - val_acc: 0.7565
Epoch 118/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5424 - acc: 0.7418 - val_loss: 0.5194 - val_acc: 0.7539
Epoch 119/150
3087/3087 [========================

...

3087/3087 [==============================] - 2s 516us/step - loss: 0.5488 - acc: 0.7389 - val_loss: 0.5230 - val_acc: 0.7448
Epoch 9/200
3087/3087 [==============================] - 2s 540us/step - loss: 0.5542 - acc: 0.7412 - val_loss: 0.5242 - val_acc: 0.7487
Epoch 10/200
3087/3087 [==============================] - 2s 520us/step - loss: 0.5498 - acc: 0.7389 - val_loss: 0.5252 - val_acc: 0.7513
Epoch 11/200
3087/3087 [==============================] - 2s 521us/step - loss: 0.5489 - acc: 0.7415 - val_loss: 0.5202 - val_acc: 0.7474
Epoch 12/200
3087/3087 [==============================] - 2s 524us/step - loss: 0.5464 - acc: 0.7415 - val_loss: 0.5223 - val_acc: 0.7526
Epoch 13/200
3087/3087 [==============================] - 2s 515us/step - loss: 0.5465 - acc: 0.7421 - val_loss: 0.5209 - val_acc: 0.7526
Epoch 14/200
3087/3087 [==============================] - 2s 511us/step - loss: 0.5440 - acc: 0.7447 - val_loss: 0.5191 - val_acc: 0.7474
Epoch 15/200
3087/3087 [========================

Epoch 66/200
3087/3087 [==============================] - 2s 506us/step - loss: 0.5387 - acc: 0.7441 - val_loss: 0.5185 - val_acc: 0.7552
Epoch 67/200
3087/3087 [==============================] - 2s 503us/step - loss: 0.5388 - acc: 0.7460 - val_loss: 0.5185 - val_acc: 0.7552
Epoch 68/200
3087/3087 [==============================] - 2s 504us/step - loss: 0.5413 - acc: 0.7457 - val_loss: 0.5186 - val_acc: 0.7539
Epoch 69/200
3087/3087 [==============================] - 2s 508us/step - loss: 0.5392 - acc: 0.7486 - val_loss: 0.5185 - val_acc: 0.7552
Epoch 70/200
3087/3087 [==============================] - 2s 510us/step - loss: 0.5399 - acc: 0.7489 - val_loss: 0.5185 - val_acc: 0.7552
Epoch 71/200
3087/3087 [==============================] - 2s 519us/step - loss: 0.5368 - acc: 0.7467 - val_loss: 0.5185 - val_acc: 0.7539
Epoch 72/200
3087/3087 [==============================] - 2s 503us/step - loss: 0.5439 - acc: 0.7460 - val_loss: 0.5185 - val_acc: 0.7539
Epoch 73/200
3087/3087 [==========

Epoch 125/200
3087/3087 [==============================] - 2s 522us/step - loss: 0.5381 - acc: 0.7464 - val_loss: 0.5185 - val_acc: 0.7526
Epoch 126/200
3087/3087 [==============================] - 2s 515us/step - loss: 0.5381 - acc: 0.7454 - val_loss: 0.5186 - val_acc: 0.7526
Epoch 127/200
3087/3087 [==============================] - 2s 503us/step - loss: 0.5353 - acc: 0.7438 - val_loss: 0.5186 - val_acc: 0.7526
Epoch 128/200
3087/3087 [==============================] - 2s 505us/step - loss: 0.5400 - acc: 0.7467 - val_loss: 0.5185 - val_acc: 0.7513
Epoch 129/200
3087/3087 [==============================] - 2s 516us/step - loss: 0.5404 - acc: 0.7489 - val_loss: 0.5186 - val_acc: 0.7526
Epoch 130/200
3087/3087 [==============================] - 2s 514us/step - loss: 0.5397 - acc: 0.7441 - val_loss: 0.5186 - val_acc: 0.7526
Epoch 131/200
3087/3087 [==============================] - 2s 510us/step - loss: 0.5393 - acc: 0.7428 - val_loss: 0.5186 - val_acc: 0.7526
Epoch 132/200
3087/3087 [==

Epoch 184/200
3087/3087 [==============================] - 2s 513us/step - loss: 0.5395 - acc: 0.7447 - val_loss: 0.5185 - val_acc: 0.7526
Epoch 185/200
3087/3087 [==============================] - 2s 570us/step - loss: 0.5397 - acc: 0.7457 - val_loss: 0.5185 - val_acc: 0.7526
Epoch 186/200
3087/3087 [==============================] - 2s 596us/step - loss: 0.5357 - acc: 0.7496 - val_loss: 0.5186 - val_acc: 0.7526
Epoch 187/200
3087/3087 [==============================] - 2s 536us/step - loss: 0.5402 - acc: 0.7447 - val_loss: 0.5185 - val_acc: 0.7526
Epoch 188/200
3087/3087 [==============================] - 2s 511us/step - loss: 0.5373 - acc: 0.7451 - val_loss: 0.5185 - val_acc: 0.7526
Epoch 189/200
3087/3087 [==============================] - 2s 510us/step - loss: 0.5368 - acc: 0.7473 - val_loss: 0.5186 - val_acc: 0.7526
Epoch 190/200
3087/3087 [==============================] - 2s 584us/step - loss: 0.5376 - acc: 0.7486 - val_loss: 0.5185 - val_acc: 0.7526
Epoch 191/200
3087/3087 [==

Epoch 42/50
3087/3087 [==============================] - 2s 517us/step - loss: 0.5452 - acc: 0.7373 - val_loss: 0.5183 - val_acc: 0.7552
Epoch 43/50
3087/3087 [==============================] - 2s 503us/step - loss: 0.5448 - acc: 0.7412 - val_loss: 0.5182 - val_acc: 0.7578
Epoch 44/50
3087/3087 [==============================] - 2s 498us/step - loss: 0.5453 - acc: 0.7386 - val_loss: 0.5181 - val_acc: 0.7578
Epoch 45/50
3087/3087 [==============================] - 2s 503us/step - loss: 0.5455 - acc: 0.7402 - val_loss: 0.5181 - val_acc: 0.7578
Epoch 46/50
3087/3087 [==============================] - 2s 500us/step - loss: 0.5498 - acc: 0.7396 - val_loss: 0.5183 - val_acc: 0.7565
Epoch 47/50
3087/3087 [==============================] - 2s 497us/step - loss: 0.5459 - acc: 0.7408 - val_loss: 0.5183 - val_acc: 0.7565
Epoch 48/50
3087/3087 [==============================] - 2s 503us/step - loss: 0.5459 - acc: 0.7421 - val_loss: 0.5182 - val_acc: 0.7565
Epoch 49/50
3087/3087 [==================

In [ ]:
0.8297